In [1]:
import os
from os import path, listdir
import tensorflow as tf
import tensorflow as tf
import cv2
import waymo_open_dataset
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
import utils
from utils import extract_and_serialize_frame, deserialize_example, extract_frame_features
import matplotlib.pyplot as plt
import numpy as np
import tarfile
import math
from PIL import Image

2022-06-19 18:33:53.807336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-06-19 18:33:53.807352: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def convert_waymo_data(tar_dir, temp_dir, tgt_dir, start, end):
    tars = [path.join(tar_dir, file) for file in listdir(tar_dir)][start:end]
    print(tars)

    for (idx, tar) in enumerate(tars):
        print(f"Extracting {tar} to {temp_dir}")
        tar_file = tarfile.open(tar, 'r:')
        tar_file.extractall(temp_dir)
        tar_file.close()

        parsed_file = path.join(tgt_dir, 'parsed-' + str(idx).rjust(3, '0') + '.tfrecord')
        license_file = f'{temp_dir}/LICENSE'

        print(f'Deleting LICENSE file {temp_dir}/LICENSE')
        if path.exists(license_file):
            os.remove(license_file)
        temp_files = [f'{temp_dir}/{file}' for file in listdir(temp_dir)]

        print(f"Parsing data and writing to {parsed_file}")
        dataset = tf.data.TFRecordDataset(temp_files)
        writer = tf.io.TFRecordWriter(parsed_file)
        for (idx, data) in enumerate(dataset):
            if idx % 100 == 0:
                print(f'Processed {idx} frames')
            converted = extract_and_serialize_frame(data)
            writer.write(converted)
        writer.close()

        print(f"Deleting temp files from {temp_dir}")
        for file in temp_files:
            os.remove(file)

In [3]:
convert_waymo_data('Data/unprocessed/training', 'Data/temp/training', 'Data/processed/training', 0, 32)
convert_waymo_data('Data/unprocessed/validation', 'Data/temp/validation', 'Data/processed/validation', 0, 8)

['Data/unprocessed/training/training_0000.tar', 'Data/unprocessed/training/training_0001.tar', 'Data/unprocessed/training/training_0002.tar', 'Data/unprocessed/training/training_0003.tar', 'Data/unprocessed/training/training_0004.tar', 'Data/unprocessed/training/training_0005.tar', 'Data/unprocessed/training/training_0006.tar', 'Data/unprocessed/training/training_0007.tar', 'Data/unprocessed/training/training_0008.tar', 'Data/unprocessed/training/training_0009.tar', 'Data/unprocessed/training/training_0010.tar', 'Data/unprocessed/training/training_0011.tar', 'Data/unprocessed/training/training_0012.tar', 'Data/unprocessed/training/training_0013.tar', 'Data/unprocessed/training/training_0014.tar', 'Data/unprocessed/training/training_0015.tar', 'Data/unprocessed/training/training_0016.tar', 'Data/unprocessed/training/training_0017.tar', 'Data/unprocessed/training/training_0018.tar', 'Data/unprocessed/training/training_0019.tar', 'Data/unprocessed/training/training_0020.tar', 'Data/unproc

2022-06-19 18:39:04.040916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-19 18:39:04.041407: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/antoinebrassardlahey/anaconda3/envs/mti830/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-06-19 18:39:04.041468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/antoinebrassardlahey/anaconda3/envs/mti830/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-06-19 1

Processed 0 frames
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Processed 500 frames
Processed 600 frames
Processed 700 frames
Processed 800 frames
Processed 900 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1700 frames
Processed 1800 frames
Processed 1900 frames
Processed 2000 frames
Processed 2100 frames
Processed 2200 frames
Processed 2300 frames
Processed 2400 frames
Processed 2500 frames
Processed 2600 frames
Processed 2700 frames
Processed 2800 frames
Processed 2900 frames
Processed 3000 frames
Processed 3100 frames
Processed 3200 frames
Processed 3300 frames
Processed 3400 frames
Processed 3500 frames
Processed 3600 frames
Processed 3700 frames
Processed 3800 frames
Processed 3900 frames
Processed 4000 frames
Processed 4100 frames
Processed 4200 frames
Processed 4300 frames
Processed 4400 frames
Processed 4500 frames


In [4]:
SCALE_PCT = 0.5

def convert_to_darknet(src_dir, tgt_dir, index_file):
    files = [path.join(src_dir, file) for file in listdir(src_dir)]
    img_id = len(listdir(f'{tgt_dir}/obj')) // 2
    
    for (idx, file) in enumerate(files):
        print(f"Converting {file} to {tgt_dir} ({idx + 1} of {len(files)})")

        dataset = tf.data.TFRecordDataset(file)
        for data in dataset:
            example = deserialize_example(data)
            og_height = example['width'].numpy()
            og_width = example['height'].numpy()
            img_width = int(og_width * SCALE_PCT)
            img_height = int(og_height * SCALE_PCT)

            image = Image.fromarray(tf.io.decode_jpeg(example['raw_image']).numpy())
            image = image.resize((img_height, img_width))

            image_name = f'example-{img_id}.jpg'
            image.save(f'{tgt_dir}/obj/{image_name}')
            
            classes = example['class'].numpy()
            boxes_center_y = example['box_center_y'].numpy()
            boxes_center_x = example['box_center_x'].numpy()
            boxes_width = example['box_width'].numpy()
            boxes_height = example['box_height'].numpy()
            label_name = f'example-{img_id}.txt'
            with open(f'{tgt_dir}/obj/{label_name}', 'w') as label_file:
                for (clazz, cx, cy, w, h) in zip(classes, boxes_center_x, boxes_center_y, boxes_height, boxes_width):
                    cx /= og_width
                    cy /= og_height
                    w /= og_width
                    h /= og_height
                    darknet_labels = f'{clazz} {cx} {cy} {w} {h}\n'
                    label_file.write(darknet_labels)
            
            index_file.write(f'data/obj/{image_name}\n')
            img_id += 1

        

In [5]:
with open('Data/darknet-data/training.txt', 'w') as training_index:
    convert_to_darknet('Data/processed/training', 'Data/darknet-data', training_index)

with open('Data/darknet-data/validation.txt', 'w') as validation_index:
    convert_to_darknet('Data/processed/validation', 'Data/darknet-data', validation_index)

Converting Data/processed/training/parsed-000.tfrecord to Data/darknet-data (1 of 32)
Converting Data/processed/training/parsed-001.tfrecord to Data/darknet-data (2 of 32)
Converting Data/processed/training/parsed-002.tfrecord to Data/darknet-data (3 of 32)
Converting Data/processed/training/parsed-003.tfrecord to Data/darknet-data (4 of 32)
Converting Data/processed/training/parsed-004.tfrecord to Data/darknet-data (5 of 32)
Converting Data/processed/training/parsed-005.tfrecord to Data/darknet-data (6 of 32)
Converting Data/processed/training/parsed-006.tfrecord to Data/darknet-data (7 of 32)
Converting Data/processed/training/parsed-007.tfrecord to Data/darknet-data (8 of 32)
Converting Data/processed/training/parsed-008.tfrecord to Data/darknet-data (9 of 32)
Converting Data/processed/training/parsed-009.tfrecord to Data/darknet-data (10 of 32)
Converting Data/processed/training/parsed-010.tfrecord to Data/darknet-data (11 of 32)
Converting Data/processed/training/parsed-011.tfreco